<h1>Sieci neuronowe - ćwiczenie 3</h1>

In [1]:
!pip install ucimlrepo


[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [75]:
def sigmoid(n) -> float:
    return 1 / (1 + np.exp(-n))

In [76]:
def sigmoid_der(n) -> float:
    return sigmoid(n) * (1 - sigmoid(n))

In [80]:
class Neuron:
    _weights: np.ndarray
    _bias: float
    _cache_x: np.ndarray
    _err: np.ndarray 

    def init_weights(self, size: int, std_dev: float):
        self._weights = np.random.normal(scale=std_dev, size=size) # initialize weights 
        self._bias = np.random.normal(scale=std_dev) #initialize bias 

    def forward(self, x: np.ndarray) -> np.ndarray:
       self._cache_x = x
       return sigmoid(np.dot(x, self._weights) + self._bias)
    
    def backward(self, error: np.ndarray):
       self._err = error * sigmoid_der(np.dot(self._cache_x, self._weights) + self._bias) 
       return self._err
    
    def backward_weights(self, error: np.ndarray, next_weights: np.ndarray):
       error = error.T @ next_weights
       self._err = error * sigmoid_der(np.dot(self._cache_x, self._weights) + self._bias) 
       return self._err

    def update_weights(self, learning_rate: float):
      self._weights = self._weights - learning_rate * np.dot(self._cache_x.T, self._err)
      self._bias = self._bias - learning_rate * np.sum(self._err)

    def get_weights(self):
       return self._weights  

    def get_bias(self):
       return self._bias  

class MultilayerNetwork:

   def __cross_entropy_loss(self, y: np.ndarray, y_pred:np.ndarray) -> np.ndarray:
      return -y*np.log(y_pred) - (1 - y)*np.log(1 - y_pred)
   
   def __cross_entropy_loss_der(self, y: np.ndarray, y_pred:np.ndarray) -> np.ndarray:
      return -np.sum(y / y_pred - (1 - y) / (1 - y_pred), axis=1)

   def __init__(self, hidden_layers_sizes = (1,) ):
      self._layers: list[list[Neuron]] = []
   
      for index, layer_size in enumerate(hidden_layers_sizes):
         self._layers.append([])
         for _ in range(layer_size):
            self._layers[index].append(Neuron()) 
      
      # output layer
      self._layers.append([])
      self._layers[-1].append(Neuron())

   def __get_weights(self) -> []:
      weights = []
      for layer in self._layers:
         weights.append([neuron.get_weights for neuron in layer])     
      return weights
    
   def __get_biases(self) -> []:
      biases = []
      for layer in self._layers:
         biases.append([neuron.get_bias for neuron in layer])     
      return biases 
   
   def __optimize(self, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray, batch_size: int, learning_rate: float, 
                        max_iter: int) -> (list, list, list, list, list, list, list, list):
      losses = []
      weights = []
      biases = []
      losses_test = []

      accuracy = []
      f_score = []
      precision = []
      recalls = []

      if batch_size > x_train.shape[0]:
         batch_size = x_train.shape[0]

      for index in range(max_iter): #learn for max_iter      
         shuffle = np.random.permutation(x_train.shape[0])
         x_train_shuffled = x_train[shuffle]
         y_train_shuffled = y_train[shuffle]

         new_train_loss = 0
         for batch_start_index in range(0, x_train.shape[0], batch_size):
            x_train_batch = x_train_shuffled[batch_start_index:batch_start_index+batch_size] 
            y_train_batch = y_train_shuffled[batch_start_index:batch_start_index+batch_size] 

            pred = self.forward(x_train_batch)
            self.backward(y_train_batch, pred)

            for layer in self._layers:
               for neuron in layer:
                  neuron.update_weights(learning_rate)
                  
         #calculate loss of training data
         new_train_loss = np.mean(self.__cross_entropy_loss(y_train_batch, pred))
         
         #calculate loss of test data
         test_pred = self.predict(x_test)
         new_test_loss = np.mean(self.__cross_entropy_loss(y_test, test_pred))

         #append to helper lists
         losses.append(new_train_loss)
         weights.append(self.__get_weights)
         biases.append(self.__get_biases)
         losses_test.append(new_test_loss)

         test_pred[test_pred >= 0.2] = 1
         test_pred[test_pred < 0.2] = 0

         #calculate scores for each iteration
         accuracy.append(metrics.accuracy_score(y_test, test_pred))
         f_score.append(metrics.f1_score(y_test, test_pred))
         precision.append(metrics.precision_score(y_test, test_pred))
         recalls.append(metrics.recall_score(y_test, test_pred))



      return losses, weights, biases, losses_test, accuracy, f_score, precision, recalls

   def logarithmicRegression(self, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray, batch_size: int, learning_rate: float, 
                       max_iter: int, std_dev: float, verb: bool = False) -> (list, list, list, list, list, list, list, list):
      
      curr_size_layer = x_train.shape[1]
      
      for layer in self._layers:
         for neuron in layer:
           neuron.init_weights(curr_size_layer, std_dev) 
         curr_size_layer = len(layer)
      
      #fix shape of y data to match further calculations
      # if len(y_test.shape) == 1:
      #    y_test = y_test[np.newaxis].T 

      # if len(y_train.shape) == 1:
      #    y_train = y_train[np.newaxis].T

      losses, weights, biases, losses_test, accuracy, f_score, precision, recalls = self.__optimize(x_train, y_train, x_test, y_test, batch_size, learning_rate, max_iter) #optimize weights and bias using training data

      if verb:
         #plot results
         plt.plot(np.arange(len(losses)), losses, label="Train Loss")
         plt.plot(np.arange(len(losses)), losses_test, label="Test loss")
         plt.legend()
         plt.show()

      #print final weight, bias, loss and scores
      print("Weights: ", self.__get_weights)
      print("Bias: ", self.__get_biases)
      print("Train loss: ", losses[-1])
      print("Test loss: ", losses_test[-1])

      print("Scores")
      print("Accuracy: ", accuracy[-1])
      print("F_score: ", f_score[-1])
      print("Precision: ", precision[-1])
      print("Recall: ", recalls[-1])

      return losses, weights, biases, losses_test, accuracy, f_score, precision, recalls      
      
   def forward(self, x: np.ndarray) -> float:
      for layer in self._layers:
         x = np.array([neuron.forward(x) for neuron in layer]).T
      return x

   def backward(self, y_train: np.ndarray, y_pred: np.ndarray) -> list[list[np.ndarray[float]]]:
      err = np.array([out_neuron.backward(self.__cross_entropy_loss_der(y_train, y_pred)) for out_neuron in self._layers[-1]])
      next_weights = np.array([out_neuron.get_weights() for out_neuron in self._layers[-1]]).T

      for layer in reversed(self._layers[:-1]):
         err = np.array([neuron.backward_weights(err, next_weights[neuron_index]) for neuron_index, neuron in enumerate(layer)])
         next_weights = np.array([neuron.get_weights() for neuron in layer]).T
         

   def predict(self, x_test: np.ndarray) -> np.ndarray:
      return self.forward(x_test) 
    


<h3>Przygotowanie danych na podstawie poprzedniego ćwiczenia</h3>

In [6]:
# original code from https://archive.ics.uci.edu/dataset/45/heart+disease
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
heart_data = heart_disease.data.original

In [42]:
df: pd.DataFrame = heart_data

# repearing of the inbalnace in classification and removing null values
df["num"] = df["num"].replace([2, 3, 4], 1) #change classes to binary classification
print(df["num"].value_counts())

#get null values of ca and remove them
null_idx = df[df["ca"].isnull()].index 
print(null_idx)
df = df.drop(null_idx)
df = df.reset_index(drop=True) 
print(df["num"].value_counts())

#get null values of thel and remove them
null_idx = df[df["thal"].isnull()].index 
print(null_idx)
df = df.drop(null_idx)
df = df.reset_index(drop=True) 
print(df["num"].value_counts())

# balance classes to same amount 138
random_idx = df.query("num == 0").sample(df["num"].value_counts()[0] - df["num"].value_counts()[1]).index 
df = df.drop(random_idx)
df = df.reset_index(drop=True)
print(df["num"].value_counts())

df_without_num = df.loc[:, df.columns != "num"]
std_features = (df_without_num - df_without_num.mean() )/ df_without_num.std() #(value-mean)/variance

result = std_features
result["heart_disease"] = df["num"]

# data not standarized 4. Danych znormalizowanych i nieznormalizowanych
n_std_result = df_without_num
n_std_result["heart_disease"] = df["num"]

result

num
0    164
1    139
Name: count, dtype: int64
Index([166, 192, 287, 302], dtype='int64')
num
0    161
1    138
Name: count, dtype: int64
Index([87, 264], dtype='int64')
num
0    160
1    137
Name: count, dtype: int64
num
0    137
1    137
Name: count, dtype: int64


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heart_disease
0,0.907760,0.658004,-2.311332,0.741248,-0.274303,2.487541,1.012907,0.077359,-0.733047,1.000794,2.201942,-0.733109,0.581532,0
1,1.352708,0.658004,0.829023,1.565972,0.745781,-0.400536,1.012907,-1.746923,1.359191,0.327454,0.597335,2.438553,-0.955508,1
2,1.352708,0.658004,0.829023,-0.633292,-0.351290,-0.400536,1.012907,-0.834782,1.359191,1.253296,0.597335,1.381332,1.093879,1
3,-1.984405,0.658004,-0.217762,-0.083476,0.052894,-0.400536,-0.998227,1.684465,-0.733047,2.010802,2.201942,-0.733109,-0.955508,0
4,0.129100,0.658004,-1.264547,-0.633292,-0.216562,-0.400536,-0.998227,1.293547,-0.733047,-0.261718,-1.007271,-0.733109,-0.955508,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,0.240337,-1.514202,0.829023,0.466340,-0.120328,-0.400536,-0.998227,-1.095394,1.359191,-0.766722,0.597335,-0.733109,1.093879,1
270,-1.094508,0.658004,-2.311332,-1.183107,0.322350,-0.400536,-0.998227,-0.704476,-0.733047,0.074952,0.597335,-0.733109,1.093879,1
271,1.463945,0.658004,0.829023,0.686266,-1.044178,2.487541,-0.998227,-0.313558,-0.733047,1.926635,0.597335,1.381332,1.093879,1
272,0.240337,0.658004,0.829023,-0.083476,-2.237484,-0.400536,-0.998227,-1.442876,1.359191,0.074952,0.597335,0.324111,1.093879,1


In [8]:
def train_test_split(features, targets, percentage):
    choices = np.random.choice(range(features.shape[0]), size=(int(features.shape[0] * percentage/100),), replace=False) 
    split = np.zeros(features.shape[0], dtype=bool)
    split[choices] = True

    return features[split], targets[split], features[~split], targets[~split]

In [73]:
features = result.loc[:, result.columns != "heart_disease"].to_numpy()
targets = result["heart_disease"].to_numpy()

x_train, y_train, x_test, y_test = train_test_split(features, targets, 70)

<h1>Testowanie modelu</h1>

In [44]:
#1. Różnej wymiarowości warstwy ukrytej
hidden1 = (4, 3)
hidden2 = (20, 10)
hidden3 = (100, 50)

#2. Różnej wartości współczynnika uczenia
learning1 = 0.001
learning2 = 0.05

#3. Różnych odchyleń standardowych przy inicjalizacji wag
std_dev = 0.8
std_dev = 0.3

#5. Różnej liczby warstw
hidden4 = (20)
hidden5 = (20, 20)
hidden6 = (20, 20, 20)

max_iter = 1000

In [81]:
ex1 = MultilayerNetwork(hidden1)
ex2 = MultilayerNetwork(hidden2)
ex3 = MultilayerNetwork(hidden3)

ex1.logarithmicRegression(x_train, y_train, x_test, y_test, 30, 0.001, max_iter, 0.4)
ex2.logarithmicRegression(x_train, y_train, x_test, y_test, 30, 0.001, max_iter, 0.4)
ex3.logarithmicRegression(x_train, y_train, x_test, y_test, 30, 0.001, max_iter, 0.4)

/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.

Weights:  <bound method MultilayerNetwork.__get_weights of <__main__.MultilayerNetwork object at 0x160b5dbd0>>
Bias:  <bound method MultilayerNetwork.__get_biases of <__main__.MultilayerNetwork object at 0x160b5dbd0>>
Train loss:  0.7386532849620239
Test loss:  0.7096940277621137
Scores
Accuracy:  0.42168674698795183
F_score:  0.0
Precision:  0.0
Recall:  0.0


/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.

Weights:  <bound method MultilayerNetwork.__get_weights of <__main__.MultilayerNetwork object at 0x160b5dc90>>
Bias:  <bound method MultilayerNetwork.__get_biases of <__main__.MultilayerNetwork object at 0x160b5dc90>>
Train loss:  0.6758954299828582
Test loss:  0.7088437241256825
Scores
Accuracy:  0.42168674698795183
F_score:  0.0
Precision:  0.0
Recall:  0.0


/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/szymon/.pyenv/versions/3.10.6/lib/python3.

In [71]:
x_train.shape[0]

191